In [23]:
#A function that we are supposed to analyze
def median(a, b, c, d, e):
    if(a > b):
        a, b = b, a
    if(c > d):
        c, d = d, c
    if(a < c):
        a = e
        if (a > b):
            a, b = b, a
    else:
        c = e
        if (c > d):
            c, d = d, c
    if(a < c):
        if b > c:
            return c
        else:
            return b
    else:
        if(d > a):
            return a
        else:
            return d

1. Construct a test suite with 5 test cases for median.py. A test suite is a set of test cases (inputs and their expected output).

In [25]:
import numpy as np
import random
# tst_suite_1= [random.randint(0,10) for i in range(5)] # [[random.randint(0,10) for i in range(5)] for j in range(5)]
tst_suite_1=(1, 2, 4, 1, 1)

2. Construct the smallest test suite that covers all the statements in median.py (~6 test cases). You can do this manually. Bonus points for doing it this automatically by using the exercise on coverage

In [26]:
#Create a fuzzer with random integers
def fuzzer():
    out = tuple(np.random.randint(10, size=5))
    return out

In [27]:
# median(*ts[1]) #tuple(list)
import sys
import inspect
from fuzzingbook.ExpectError import ExpectTimeout

coverage = []
def traceit(frame, event, arg):
    if event == "line":
        global coverage
        function_name = frame.f_code.co_name
        lineno = frame.f_lineno
        coverage.append(lineno)
    return traceit

def runner():
    global coverage
    tst_suite_2 = []
    covered_lines = {}
    code = inspect.getsource(median)
    code_lines = [""] + code.splitlines()
    while (len(covered_lines) < len(code_lines)-6):
        #lines 0,1,11,18,20 won't be registered for traceit
        inputs = fuzzer()
        tst_suite_2.append(inputs)
        coverage = []
        sys.settrace(traceit)  
        run = median(inputs[0], inputs[1], inputs[2], inputs[3], inputs[4])
        sys.settrace(None)
        coverage = set(coverage)
        covered_lines = coverage.union(covered_lines)
    print("Covered lines: {}".format(covered_lines))
    print("Test suite: ", tst_suite_2)
    return tst_suite_2

tst_suite_2 = runner()
tst_suite_2 = [(2,1,1,1,1), (1,2,4,1,1), (1,1,2,3,1), (1,1,1,2,1), (1,1,1,1,2), (1,1,1,1,1), (2,1,3,4,5)]
# code = inspect.getsource(median)
# code_lines = [""] + code.splitlines()
# for i in range(0,len(code_lines)):
#     print("{} {}".format(i, code_lines[i]))


Covered lines: {2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 18, 20, 21, 23}
Test suite:  [(2, 1, 6, 8, 8), (4, 8, 9, 2, 0), (5, 3, 4, 0, 7), (3, 4, 9, 4, 4), (5, 7, 3, 3, 8), (9, 8, 5, 5, 8), (1, 3, 9, 4, 5), (3, 1, 4, 2, 6), (9, 1, 1, 5, 4), (6, 3, 0, 8, 2), (5, 5, 9, 0, 8)]


3. Manually mutate median.py to inject bugs and run both test suites in 1 and 2. A good test suite should be able to detect as many bugs as possible. (see mutation score in mutation analysis chapter). Try to introduce one bug at a time to see the quality of your test suites. Create at least 10 mutants. What is the mutation score for each test suite. 

In [29]:
#(1, 2, 4, 1, 1)
def faulty_median(a, b, c, d, e):
    if(a < b): #Injected fault < in place of >
        a, b = b, e  #Injected fault e in place of a
    if(c > d):
        c, d = 1, 1 #Injected fault 1 in place of  d,c
    if(a < c):
        a = a #Injected fault a in place of e
        if (a < b): #Injected fault < in place of a > b
            a, b = b, a 
    else:
        c > e # Injected fault > in place of =
        if (c < d): #Injected fault  < in place of >
            c, d = d, c
    if(a < c):
        if b > c:
            return c
        else:
            return a #Injected fault None in place of return c
    else:
        if(d > a):
            return None #Injected fault None in place of return a
        else:
            pass #Injected fault pass in place of return d

In [38]:
#Here are a few test cases to ensure that the program works.
def weak_oracle(fmedian):
    assert fmedian(*tst_suite_1) == median(*tst_suite_1)

def strong_oracle(fmedian):
    for i in tst_suite_2:
        assert fmedian(*i) == median(*i) #assert median(1, 2, 4, 1, 1) == 1

In [39]:
#Check if our test-suites are able to detect fault
from fuzzingbook.MutationAnalysis import ExpectError

In [40]:
#For test-suite 1
with ExpectError():
    weak_oracle(faulty_median)
print("Score is:", 1/10)

Score is: 0.1


Traceback (most recent call last):
  File "<ipython-input-40-8731dba7778e>", line 3, in <module>
    weak_oracle(faulty_median)
  File "<ipython-input-38-b5a8bf67ff86>", line 3, in weak_oracle
    assert fmedian(*tst_suite_1) == median(*tst_suite_1)
AssertionError (expected)


In [41]:
# from fuzzingbook.MutationAnalysis import Coverage
# with Coverage() as cov:
#     weak_oracle(median)
# cov.show_coverage(median)

In [43]:
with ExpectError():
    strong_oracle(faulty_median)
print("Score is:", 2/10)

Score is: 0.2


Traceback (most recent call last):
  File "<ipython-input-43-5ac8d0388df4>", line 2, in <module>
    strong_oracle(faulty_median)
  File "<ipython-input-38-b5a8bf67ff86>", line 7, in strong_oracle
    assert fmedian(*i) == median(*i) #assert median(1, 2, 4, 1, 1) == 1
AssertionError (expected)


Practically it is not efficient to do step 3 manually. Write python code to automate the process. And for each mutant show how it's different from the original.

In [44]:
import ast
import astor
import inspect

In [45]:
#The PMIterator, which is the iterator class for MuFunctionAnalyzer
class PMIterator():
    def __init__(self, pm):
        self.pm = pm
        self.idx = 0
        
    def __next__(self): #The next() method returns the corresponding Mutant
        i = self.idx
        if i >= self.pm.nmutations:
            self.pm.finish()
            raise StopIteration()
        self.idx += 1
        mutant = Mutant(self.pm, self.idx, log=self.pm.log)
        self.pm.register(mutant)
        return mutant
    
#Class for Mutation Analysis of test suite
class MuFunctionAnalyzer():
    def __init__(self, fn, log=False):
        self.fn = fn
        self.name = fn.__name__
        src = inspect.getsource(fn)
        self.ast = ast.parse(src)
        self.src = astor.to_source(self.ast)  # normalize
        self.mutator = self.mutator_object()
        self.nmutations = self.get_mutation_count()
        self.un_detected = set()
        self.mutants = []
        self.log = log

    def mutator_object(self, locations=None):
        return StmtDeletionMutator(locations)

    def register(self, m):
        self.mutants.append(m)

    def finish(self):
        self.un_detected = {mutant for mutant in self.mutants if not mutant.detected}
    
    def get_mutation_count(self): #fun gives no. of possible mutations
        self.mutator.visit(self.ast)
        return self.mutator.count
    
    def __iter__(self):
        return PMIterator(self)
    
    def score(self):
        return (self.nmutations - len(self.un_detected)) / self.nmutations

In [46]:
#The Mutator provides the base class for implementing individual mutations.
class Mutator(ast.NodeTransformer):
    def __init__(self, mutate_location=-1):
        self.count = 0
        self.mutate_location = mutate_location

    def mutable_visit(self, node):
        self.count += 1  # statements start at line no 1
        if self.count == self.mutate_location:
            return self.mutation_visit(node)
        return self.generic_visit(node)

    
#Replaces given stmt with pass. It performs mutation by replacing the given statement with pass
class StmtDeletionMutator(Mutator):
    def visit_Return(self, node): return self.mutable_visit(node)
    def visit_Delete(self, node): return self.mutable_visit(node)

    def visit_Assign(self, node): return self.mutable_visit(node)
    def visit_AnnAssign(self, node): return self.mutable_visit(node)
    def visit_AugAssign(self, node): return self.mutable_visit(node)

    def visit_Raise(self, node): return self.mutable_visit(node)
    def visit_Assert(self, node): return self.mutable_visit(node)

    def visit_Global(self, node): return self.mutable_visit(node)
    def visit_Nonlocal(self, node): return self.mutable_visit(node)

    def visit_Expr(self, node): return self.mutable_visit(node)

    def visit_Pass(self, node): return self.mutable_visit(node)
    def visit_Break(self, node): return self.mutable_visit(node)
    def visit_Continue(self, node): return self.mutable_visit(node)

    #Replaces given stmt with pass. actual mutation consists of replacing the node with a pass statement:
class StmtDeletionMutator(StmtDeletionMutator):
    def mutation_visit(self, node): return ast.Pass()  

In [47]:
MuFunctionAnalyzer(median).nmutations #For median, this visitor produces five mutations – namely, replacing the five return statements with pass

10

In [48]:
#The Mutant class contains logic for generating mutants when given the locations to mutate.
class Mutant():
    def __init__(self, pm, location, log=False):
        self.pm = pm
        self.i = location
        self.name = "%s_%s" % (self.pm.name, self.i)
        self._src = None
        self.tests = []
        self.detected = False
        self.log = log    
        
    def generate_mutant(self, location):
        mutant_ast = self.pm.mutator_object(location).visit(ast.parse(self.pm.src))  # copy
        return astor.to_source(mutant_ast)
    
    def src(self):
        if self._src is None:
            self._src = self.generate_mutant(self.i)
        return self._src

    def __enter__(self): 
        #It creates the mutant as a function & places it in global namespace, so that 
        #the assert statement executes the mutated function rather than the original.
        if self.log:
            print('->\t%s' % self.name)
        c = compile(self.src(), '<mutant>', 'exec')
        eval(c, globals())
    
    def __exit__(self, exc_type, exc_value, traceback):
        #The __exit__() function checks whether an exception has occurred
        #if so, it marks the mutation as detected. Finally, it restores the original function definition.
        if self.log:
            print('<-\t%s' % self.name)
        if exc_type is not None:
            self.detected = True
            if self.log:
                print("Detected %s" % self.name, exc_type, exc_value)
        globals()[self.pm.name] = self.pm.fn
        if self.log:
            print()
        return True

In [50]:
for mutant in MuFunctionAnalyzer(faulty_median, log=True):
    with mutant:
        weak_oracle(faulty_median)
print("\n\nThe score is: " ,mutant.pm.score())

->	faulty_median_1
<-	faulty_median_1
Detected faulty_median_1 <class 'AssertionError'> 

->	faulty_median_2
<-	faulty_median_2
Detected faulty_median_2 <class 'AssertionError'> 

->	faulty_median_3
<-	faulty_median_3
Detected faulty_median_3 <class 'AssertionError'> 

->	faulty_median_4
<-	faulty_median_4
Detected faulty_median_4 <class 'AssertionError'> 

->	faulty_median_5
<-	faulty_median_5
Detected faulty_median_5 <class 'AssertionError'> 

->	faulty_median_6
<-	faulty_median_6
Detected faulty_median_6 <class 'AssertionError'> 

->	faulty_median_7
<-	faulty_median_7
Detected faulty_median_7 <class 'AssertionError'> 

->	faulty_median_8
<-	faulty_median_8
Detected faulty_median_8 <class 'AssertionError'> 

->	faulty_median_9
<-	faulty_median_9
Detected faulty_median_9 <class 'AssertionError'> 

->	faulty_median_10
<-	faulty_median_10
Detected faulty_median_10 <class 'AssertionError'> 



The score is:  1.0


In [51]:
def triangle(a, b, c):
    if a == b:
        if b == c:
            return 'Equilateral'
        else:
            return 'Isosceles'
    else:
        if b == c:
            return "Isosceles"
        else:
            if a == c:
                return "Isosceles"
            else:
                return "Scalene"
#basic test function
def strong_oracle1(fn):
    assert fn(1, 1, 1) == 'Equilateral'
    assert fn(1, 2, 1) == 'Isosceles'
    assert fn(2, 2, 1) == 'Isosceles'
    assert fn(1, 2, 2) == 'Isosceles'
    assert fn(1, 2, 3) == 'Scalene'
    
for mutant in MuFunctionAnalyzer(triangle, log=True):
    with mutant:
        strong_oracle1(triangle)
print("\n\nThe score is: " ,mutant.pm.score())

->	triangle_1
<-	triangle_1
Detected triangle_1 <class 'AssertionError'> 

->	triangle_2
<-	triangle_2
Detected triangle_2 <class 'AssertionError'> 

->	triangle_3
<-	triangle_3
Detected triangle_3 <class 'AssertionError'> 

->	triangle_4
<-	triangle_4
Detected triangle_4 <class 'AssertionError'> 

->	triangle_5
<-	triangle_5
Detected triangle_5 <class 'AssertionError'> 



The score is:  1.0


In [19]:
import difflib
for mutant in MuFunctionAnalyzer(median):
    shape_src = mutant.pm.src
    for line in difflib.unified_diff(mutant.pm.src.split('\n'),
                                     mutant.src().split('\n'),
                                     fromfile=mutant.pm.name,
                                     tofile=mutant.name, n=3):
        print(line)

--- median

+++ median_1

@@ -1,6 +1,6 @@

 def median(a, b, c, d, e):
     if a > b:
-        a, b = b, a
+        pass
     if c > d:
         c, d = d, c
     if a < c:
--- median

+++ median_2

@@ -2,7 +2,7 @@

     if a > b:
         a, b = b, a
     if c > d:
-        c, d = d, c
+        pass
     if a < c:
         a = e
         if a > b:
--- median

+++ median_3

@@ -4,7 +4,7 @@

     if c > d:
         c, d = d, c
     if a < c:
-        a = e
+        pass
         if a > b:
             a, b = b, a
     else:
--- median

+++ median_4

@@ -6,7 +6,7 @@

     if a < c:
         a = e
         if a > b:
-            a, b = b, a
+            pass
     else:
         c = e
         if c > d:
--- median

+++ median_5

@@ -8,7 +8,7 @@

         if a > b:
             a, b = b, a
     else:
-        c = e
+        pass
         if c > d:
             c, d = d, c
     if a < c:
--- median

+++ median_6

@@ -10,7 +10,7 @@

     else:
         c = e
         if c > d:
-            c, 